# YouTube Comment Cleaning and Exploration

Takes data from a CSV file and cleans the data to isolate comments of interest

In [1]:
import re
import nagisa
import pandas as pd
from collections import Counter
from functools import reduce
from operator import add
from pathlib import Path

NumExpr defaulting to 8 threads.


In [2]:
input_path = Path('Resources/youtube_comments.csv')
comments_df = pd.read_csv(input_path, index_col=0)

comments_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type
0,mwamjapan,Jb6Zlg30rgk,10,This season is going to be a masterpiece <3,2023-04-16T15:39:10Z,top-level
1,mwamjapan,Jb6Zlg30rgk,10,Every season is masterpiece 🔥🔥🔥,2023-04-17T16:41:52Z,reply
2,mwamjapan,Jb6Zlg30rgk,10,@HM cry about it,2023-04-17T16:15:19Z,reply
3,mwamjapan,Jb6Zlg30rgk,10,@HM dude demon slayer has no story but it has ...,2023-04-17T15:44:29Z,reply
4,mwamjapan,Jb6Zlg30rgk,10,@HM 🙂🙃😒😒😒,2023-04-17T13:44:05Z,reply
5,mwamjapan,Jb6Zlg30rgk,10,Honestamente no entiendo porqué esta canción r...,2023-04-17T03:58:25Z,top-level
6,mwamjapan,Jb6Zlg30rgk,10,​@みっくん𓈒𓂂◌𝙼𝙸𝚈𝚄 llora pues,2023-04-17T17:15:35Z,reply
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply
8,mwamjapan,Jb6Zlg30rgk,10,悩めば、イイ！,2023-04-17T14:53:19Z,reply
9,mwamjapan,Jb6Zlg30rgk,10,@みっくん𓈒𓂂◌𝙼𝙸𝚈𝚄 ニワカほどそう言うよね！,2023-04-17T13:11:13Z,reply


In [3]:
# RE patterns needed for JP text
hiragana = r'\u3041-\u3096'
katakana = r'\u30A1-\u30F6'
kanji = r'\u3006\u4E00-\u9FFF'

# Matches to test for existence of text that uses a Japanese script
jp_text = rf'[{hiragana}{katakana}{kanji}]'

# Accepted characters
accepted_char = rf'[a-zA-Z0-9\u30FC\u3005{hiragana}{katakana}{kanji}]'

In [4]:
# Remove @username from replies
# Done before rest of cleaning to ensure a user using a Japanese script doesn't pass filter
comments_df['text'] = comments_df['text'].str.replace(r'@\S+\s', '', regex=True)

In [5]:
# Filter out comments that don't have any Japanese characters
has_jp_char = comments_df.loc[comments_df['text'].str.contains(jp_text, regex=True)]

has_jp_char.sample(10)

,channel,video_id,category_id,text,date_published,comment_type
23,Uru Official YouTube Channel,x60xR0TEQ88,10,曲調や歌詞、世界観、Uruさんの全てが詰まってる。本当に素敵です。,2023-04-15T23:10:45Z,top-level
84,KAI Channel / 朝倉海,frMtfMxTVT4,24,女子枠なると審査員バカなるのすきw,2023-04-13T12:47:50Z,top-level
70,社畜OLちえ丸,kI98lHMx15o,22,プライベートサウナめっちゃ行きたくなりました！\n久しぶりにちえ丸さんの動画見たけど、やっぱ...,2023-04-17T10:58:46Z,top-level
67,BUMP OF CHICKEN,xfU9eYMKbDE,10,このあいだ初めてliveに行ったのですが、自分が音響や電飾に過敏に反応してしまう体質らしく、...,2023-04-13T04:04:56Z,top-level
51,Kenshi Yonezu 米津玄師,QeqbN1IZVpk,10,なんて柔らかなお声…。米津玄師様のご活躍ももう１５年近くにも及ぶのですね…。私はそのうち、た...,2023-04-09T13:42:16Z,top-level
75,Kenshi Yonezu 米津玄師,QeqbN1IZVpk,10,日常の倦怠感も全てひっくるめて、見せ方・聞かせ方・魅了の仕方、米津さんはエンターティナーだな...,2023-04-10T08:45:57Z,top-level
43,もちまる日記,AbFPtDp-3bc,15,もち様の鳴き声って、なんて可愛らしいんでしょう。,2023-04-11T11:31:29Z,top-level
58,THE RAMPAGE from EXILE TRIBE,cnxtRkvWG-M,10,ほんとに最高。\nレースでもありゲームでもある、独特な世界観。\n見ていて引き込まれる。\n...,2023-04-14T05:48:16Z,top-level
104,ちゃんねる鰐,urW-CCjGxVw,15,いろんな人にいろんな事言われても鰐さんが良いと思うやり方でやっていければそれがいちばんな気が...,2023-04-13T14:01:26Z,top-level
8,タガヤスのポケモン,XFGUyu7Fkdc,20,毎度ながら、視聴者を飽きさせないようにと挟む小ネタ的なヤツがこれでもかと言うほど力入っててす...,2023-04-17T13:37:03Z,top-level


In [6]:
# Remove other unnecessary characters - 
def clean_text(text):
    
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\t', '', text)
    text = re.sub(r'\r', '', text)
    text = re.sub(r'https?://[a-zA-Z0-9.-]*[/?[a-zA-Z0-9.-_]*]*', '', text)
    text = re.sub(r'笑+', '笑', text)
    
    filtered_text = ''
    
    for char in text:
        if re.match(accepted_char, char):
            
            filtered_text += char
            
    return filtered_text

In [7]:
# Clean
cleaned_df = has_jp_char.copy()
cleaned_df['text'] = cleaned_df['text'].apply(clean_text)

cleaned_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply
8,mwamjapan,Jb6Zlg30rgk,10,悩めばイイ,2023-04-17T14:53:19Z,reply
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど声の相性が良くて特に重なった時に何とも言えない不...,2023-04-17T00:54:46Z,top-level
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで作中にはいろんな絆ノ奇跡がありますよね特に刀鍛冶の里編のラストは絆ノ奇跡その...,2023-04-17T13:48:13Z,reply
46,mwamjapan,Jb6Zlg30rgk,10,我が命果てようとも繋いでいこう自分が志半ばで死んでも繋いだ命がきっと果たしてくれると二話で炭...,2023-04-17T13:37:24Z,reply
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました,2023-04-17T13:07:22Z,reply
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply


In [8]:
# Check size of DF
cleaned_df.shape

(11297, 6)

In [9]:
# Check for null values
cleaned_df.isna().sum()

channel           0
video_id          0
category_id       0
text              0
date_published    0
comment_type      0
dtype: int64

In [10]:
# Check unique values in each
cleaned_df.nunique()

channel              82
video_id             92
category_id          12
text              10659
date_published    11104
comment_type          2
dtype: int64

In [11]:
# Check amounts of top-level vs. replies for comments
cleaned_df['comment_type'].value_counts()

top-level    8320
reply        2977
Name: comment_type, dtype: int64

In [13]:
# Tokenize text
analysis_df = cleaned_df.copy()
analysis_df['tokens'] = analysis_df['text'].apply(lambda x: nagisa.tagging(x))

analysis_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type,tokens
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply,たか/名詞 が/助詞 OP/名詞 で/助詞 世界/名詞 観/接尾辞 が/助詞 壊れる/動詞 ...
8,mwamjapan,Jb6Zlg30rgk,10,悩めばイイ,2023-04-17T14:53:19Z,reply,悩め/動詞 ば/助詞 イイ/形容詞
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply,ニワカ/形状詞 ほど/助詞 そう/副詞 言う/動詞 よ/助詞 ね/助詞
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply,珍しく/形容詞 ショート/名詞 動画/名詞 が/助詞 出/動詞 て/助動詞 ます/助動詞 ね...
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply,めっちゃ/副詞 同意/名詞
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど声の相性が良くて特に重なった時に何とも言えない不...,2023-04-17T00:54:46Z,top-level,milet/名詞 と/助詞 の/助詞 コラボ/名詞 に/助詞 は/助詞 最初/名詞 びっくり...
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで作中にはいろんな絆ノ奇跡がありますよね特に刀鍛冶の里編のラストは絆ノ奇跡その...,2023-04-17T13:48:13Z,reply,全く/副詞 仰る/動詞 とおり/名詞 で/助詞 作中/名詞 に/助詞 は/助詞 いろんな/連...
46,mwamjapan,Jb6Zlg30rgk,10,我が命果てようとも繋いでいこう自分が志半ばで死んでも繋いだ命がきっと果たしてくれると二話で炭...,2023-04-17T13:37:24Z,reply,我が/連体詞 命/名詞 果てよう/動詞 と/助詞 も/助詞 繋い/動詞 で/助詞 いこう/動...
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました,2023-04-17T13:07:22Z,reply,我が/連体詞 命/名詞 果てよう/動詞 とも/名詞 に/助詞 は/助詞 無一郎/名詞 を/助...
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply,鬼滅/名詞 の/助詞 刃/名詞 に/助詞 は/助詞 色んな/連体詞 絆ノ/名詞 奇跡/名詞 ...


In [14]:
# Created filtered column
analysis_df['filtered_tokens'] = (analysis_df['text']
                                  .apply(lambda x: nagisa.filter(x, filter_postags=['助詞', '助動詞'])))

analysis_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type,tokens,filtered_tokens
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply,たか/名詞 が/助詞 OP/名詞 で/助詞 世界/名詞 観/接尾辞 が/助詞 壊れる/動詞 ...,たか/名詞 OP/名詞 世界/名詞 観/接尾辞 壊れる/動詞 アニメ/名詞 ない/形容詞 は...
8,mwamjapan,Jb6Zlg30rgk,10,悩めばイイ,2023-04-17T14:53:19Z,reply,悩め/動詞 ば/助詞 イイ/形容詞,悩め/動詞 イイ/形容詞
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply,ニワカ/形状詞 ほど/助詞 そう/副詞 言う/動詞 よ/助詞 ね/助詞,ニワカ/形状詞 そう/副詞 言う/動詞
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply,珍しく/形容詞 ショート/名詞 動画/名詞 が/助詞 出/動詞 て/助動詞 ます/助動詞 ね...,珍しく/形容詞 ショート/名詞 動画/名詞 出/動詞 最高/名詞 すぎる/動詞
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply,めっちゃ/副詞 同意/名詞,めっちゃ/副詞 同意/名詞
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど声の相性が良くて特に重なった時に何とも言えない不...,2023-04-17T00:54:46Z,top-level,milet/名詞 と/助詞 の/助詞 コラボ/名詞 に/助詞 は/助詞 最初/名詞 びっくり...,milet/名詞 コラボ/名詞 最初/名詞 びっくり/名詞 し/動詞 声/名詞 相性/名詞 ...
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで作中にはいろんな絆ノ奇跡がありますよね特に刀鍛冶の里編のラストは絆ノ奇跡その...,2023-04-17T13:48:13Z,reply,全く/副詞 仰る/動詞 とおり/名詞 で/助詞 作中/名詞 に/助詞 は/助詞 いろんな/連...,全く/副詞 仰る/動詞 とおり/名詞 作中/名詞 いろんな/連体詞 絆ノ/名詞 奇跡/名詞 ...
46,mwamjapan,Jb6Zlg30rgk,10,我が命果てようとも繋いでいこう自分が志半ばで死んでも繋いだ命がきっと果たしてくれると二話で炭...,2023-04-17T13:37:24Z,reply,我が/連体詞 命/名詞 果てよう/動詞 と/助詞 も/助詞 繋い/動詞 で/助詞 いこう/動...,我が/連体詞 命/名詞 果てよう/動詞 繋い/動詞 いこう/動詞 自分/名詞 志/名詞 半ば...
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました,2023-04-17T13:07:22Z,reply,我が/連体詞 命/名詞 果てよう/動詞 とも/名詞 に/助詞 は/助詞 無一郎/名詞 を/助...,我が/連体詞 命/名詞 果てよう/動詞 とも/名詞 無一郎/名詞 思い浮かべ/動詞
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply,鬼滅/名詞 の/助詞 刃/名詞 に/助詞 は/助詞 色んな/連体詞 絆ノ/名詞 奇跡/名詞 ...,鬼滅/名詞 刃/名詞 色んな/連体詞 絆ノ/名詞 奇跡/名詞 あり/動詞


In [15]:
# Use for examining properties of the tokens
test = analysis_df.iloc[0]['filtered_tokens']
test

In [16]:
test.words

['たか', 'OP', '世界', '観', '壊れる', 'アニメ', 'ない', 'はげ', 'たこ']

In [17]:
type(test.words[0])

str

In [18]:
str(test)

'たか/名詞 OP/名詞 世界/名詞 観/接尾辞 壊れる/動詞 アニメ/名詞 ない/形容詞 はげ/名詞 たこ/名詞'

In [20]:
# Extract filtered_tokens into a list
def extract_tokens(df_column):

    token_list = df_column.apply(lambda x: str(x).split()).tolist()
    word_list = reduce(add, token_list)
    
    return word_list

word_list = extract_tokens(analysis_df['filtered_tokens'])

In [21]:
# Get total word count and examine sample of the list
total_words = len(word_list)
print(total_words, word_list[:10])

163992 ['たか/名詞', 'OP/名詞', '世界/名詞', '観/接尾辞', '壊れる/動詞', 'アニメ/名詞', 'ない/形容詞', 'はげ/名詞', 'たこ/名詞', '悩め/動詞']


In [31]:
# Count and sort by occurrences, display as a DF
def count_words(word_list):
    
    counter = Counter(word_list)
    freq_list = counter.most_common()
    
    return freq_list

In [32]:
# Organize data as a DF display
def display_freq(freq_list):
    
    freq_df = pd.DataFrame.from_records(list(dict(most_occurrences).items()),
                                        columns=['word', 'count'])
    freq_df[['word', 'pos']] = freq_df['word'].str.split('/', expand=True)
    freq_df = freq_df[['word', 'pos', 'count']]
    
    return freq_df

In [33]:
# Calculating totals for entire dataset
freq_list = count_words(word_list)
freq_df = display_freq(freq_list)

freq_df.head(10)

,word,pos,count
0,し,動詞,3578
1,1,名詞,2347
2,さん,接尾辞,2149
3,0,名詞,2094
4,2,名詞,1848
5,3,名詞,1492
6,お,接頭辞,1439
7,5,名詞,1255
8,4,名詞,1204
9,ちゃん,接尾辞,1182


In [36]:
# Remove numbers from words - this data is uninteresting
freq_df = freq_df.loc[~freq_df['word'].str.contains('\d+')]

freq_df.head(25)

,word,pos,count
0,し,動詞,3578
2,さん,接尾辞,2149
6,お,接頭辞,1439
9,ちゃん,接尾辞,1182
10,見,動詞,1096
11,この,連体詞,844
12,ありがとう,感動詞,813
13,いい,形容詞,796
14,好き,形状詞,788
15,ない,形容詞,786


In [38]:
freq_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17523 entries, 0 to 17630
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    17523 non-null  object
 1   pos     17523 non-null  object
 2   count   17523 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 547.6+ KB


In [39]:
def get_percentages(freq_df):
    
    total_words = sum(freq_df['count'])
    freq_df['freq_percentage'] = freq_df['count'] / total_words
    
    return freq_df

In [40]:
freq_df = get_percentages(freq_df)

freq_df.head(25)

,word,pos,count,freq_percentage
0,し,動詞,3578,0.023613
2,さん,接尾辞,2149,0.014182
6,お,接頭辞,1439,0.009497
9,ちゃん,接尾辞,1182,0.007800
10,見,動詞,1096,0.007233
11,この,連体詞,844,0.005570
12,ありがとう,感動詞,813,0.005365
13,いい,形容詞,796,0.005253
14,好き,形状詞,788,0.005200
15,ない,形容詞,786,0.005187
